In [ ]:
import pandas as pd
import json
import plotly.express as px

In [ ]:
if "snakemake" in locals():
    input_departures_path = snakemake.input["departures"]
    input_arrivals_path = snakemake.input["arrivals"]
    input_date = snakemake.params["date"]

    output_path = snakemake.output[0]
else:
    input_departures_path = "../../../../resources/brussels/airport/2023.Departure.aggregate.xlsx"
    input_arrivals_path = "../../../../resources/brussels/airport/2023.Arrival.aggregate.xlsx"
    input_date = "max"
    
    output_path = "../../../../results/brussels/airport/daily_totals.parquet"

In [ ]:
df_departures = pd.read_excel(input_departures_path)
df_departures = df_departures.rename(columns = { "Date": "date", "Passenger": "passengers" })
df_departures["direction"] = "departures"

df_arrivals = pd.read_excel(input_arrivals_path)
df_arrivals = df_arrivals.rename(columns = { "Date": "date", "Passenger": "passengers" })
df_arrivals["direction"] = "arrivals"

df_daily = pd.concat([df_departures, df_arrivals])

In [ ]:
px.line(
    df_daily, x = "date", y = "passengers", color = "direction",
    title = "Daily passengers at Brussels airport"
)

In [ ]:
df_daily = df_daily.pivot(index = ["date"], columns = ["direction"])
df_daily.columns = ["arrivals", "departures"]
df_daily = df_daily.reset_index()
df_daily["total"] = df_daily["arrivals"] + df_daily["departures"]

In [ ]:
if input_date == "max":
    df_result = df_daily.sort_values(by = "total", ascending = False).iloc[:1]
else:
    df_result = df_daily[df_daily["date"] == input_date]

assert len(df_result) == 1
df_result

In [ ]:
df_result = pd.DataFrame({
    "is_access": [False, True],
    "count": [df_result["arrivals"].values[0], df_result["departures"].values[0]]
})

df_result["weight"] = df_result["count"] / df_result["count"].sum()
df_result

In [ ]:
df_result.to_parquet(output_path)